In [13]:
from bs4 import BeautifulSoup
import pandas as pd
import os

In [77]:
### SET GLOBAL VARIABLES ###

main_id = "A445182"   # Put your GED Match ID here as a string
cm_threshold = 20         # Set the minimum CM shared value to include in csv files
directory = "../Gephi2024/Import_Files/"    # Set directory to write csv output to

# Insert a list of ged_ids you want to include in nodes and edges
ged_ids = ["QL6379034", "YG9756632", "A637470", "WP4802642", "BA8594640", "A195755", "A927216", "BQ8038683", "A199479",
          "FY4162947", "A398870", "YM3561700", "RQ3314375", "A684443", "T626480", "ZE4463791", "YG9844517", "FS026156C1",
          "H416083", "A227609", "A662421", "T719130", "A313175", "A865831", "BK7964209", "CB6148485", "EP5453655", "A591144",
          "A265533", "T391930", "M104341", "A943947", "M741893", "UR5098028", "TF1360118", "A876924", "FT9172213", "DM1066832",
          "M384577", "LR9426891", "M594527", "M464691", "MM2289940", "T596736", "A213723", "EY2933526", "DS5487294", "JS8796188",
          "A429925", "QQ5823295", "T377862", "A245559", "A455770", "A955088", "A115797", "YJ1893714", "T236915", "A944831",
          "QL6379034", "ZC7060781", "A582117", "A195588", "A083036", "HA9368773", "A718760", "XF3426993", "A268000", "A854177",
          "T667513", "AZ6213436", "LD6428144", "PG6174337", "A024726", "A355405", "FU2571459", "LK8998008", "MJ3959069", "A046943",
          "M120933", "A005048", "M490247", "A699548", "SP8745883", "A427327", "MF9506137", "RM1409135", "T538957", "SD4763449",
          "FA1293905", "QX2202465", "M687594", "GM3784447", "AZ2693023", "M414013", "CA7688039", "XX7320466", "XE3446978", "GT2163037",
          "T529415", "A366378", "MU4742441", "A627140", "LF5490888", "A459788", "KC4549308", "HB6603003", "A680411", "A007587", "SY3556612",
          "SW9841654", "A078377", "A380565", "A311042", "A324230", "A704422", "WR5697457", "BV6192805", "A725191", "A050279", "KP7709976",
          "ZW1338832"]

edges = pd.DataFrame()
nodes = pd.DataFrame()

In [78]:
### CREATE OR RESET CSV FILES ###

edge_header = pd.DataFrame(["Source","Target","Type", "Weight", "Shared_CM"]).T
node_header = pd.DataFrame(["ID","Name","Direction", "Weight", "Shared_CM", "Tree", "Email"]).T

edge_header.to_csv(f"{directory}edges.csv", header=False, index=False)
node_header.to_csv(f"{directory}nodes.csv", header=False, index=False)

In [79]:
### Read shared_master.csv into dataframe

df = pd.read_csv(f"{directory}shared_master.csv")

In [80]:
df.shape

(35535, 13)

In [81]:
### Create a file to check for html downloads

df2 = df["match_id"].unique()

df2 = pd.DataFrame(df2)
df2[1] = "in_master"

for ged_id in ged_ids:
    if ged_id in df["match_id"].unique():
        print(f"{ged_id} is in master_share")
    else:
        print(f"{ged_id} is not in master_share")
        
df2.to_csv(f"{directory}vlookupfile.csv", header=False, index=False)

QL6379034 is in master_share
YG9756632 is in master_share
A637470 is in master_share
WP4802642 is in master_share
BA8594640 is in master_share
A195755 is in master_share
A927216 is in master_share
BQ8038683 is in master_share
A199479 is in master_share
FY4162947 is in master_share
A398870 is in master_share
YM3561700 is in master_share
RQ3314375 is in master_share
A684443 is in master_share
T626480 is in master_share
ZE4463791 is in master_share
YG9844517 is in master_share
FS026156C1 is in master_share
H416083 is in master_share
A227609 is in master_share
A662421 is in master_share
T719130 is in master_share
A313175 is in master_share
A865831 is in master_share
BK7964209 is in master_share
CB6148485 is in master_share
EP5453655 is in master_share
A591144 is in master_share
A265533 is in master_share
T391930 is in master_share
M104341 is in master_share
A943947 is in master_share
M741893 is in master_share
UR5098028 is in master_share
TF1360118 is in master_share
A876924 is in master_s

In [82]:
### Create nodes and edges csv ###


df = df.query("main_cm >= @cm_threshold")


# Add new columns
df["type"] = "Undirected"
df["shared_cm"] = df["main_cm"]
    
# Iterate through each html file using ged_ids
for ged_id in ged_ids:
    match_id = ged_id
    
    df3 = df.query("main_id == @main_id and match_id == @match_id")
    print(f"{main_id} and {match_id}")
    
### CLEAN AND PREPARE DATAFRAME ###
    
    #Filter shared_master dataframe down by ged_ids and threshold
    


### CREATE NODES AND EDGES DATAFRAMES ###   
    
    # Create edge dataframes and then concat into one dataframe
    edge1 = df3.loc[:, ["main_id","target","type", "main_gen", "main_cm"]]
    
    edge2 = df3.loc[:,["match_id", "target", "type", "match_gen", "match_cm"]]
    edge2.columns =["main_id","target", "type", "main_gen","main_cm"]
   

    edges_temp = pd.concat([edge1, edge2])
    
    # Create node dateframe
    nodes_temp = (df.loc[:,["target", "name", "type", "main_cm", "shared_cm", "tree", "email"]])

    edges = pd.concat([edges, edges_temp])
    nodes = pd.concat([nodes, nodes_temp])
    
    print(f"Appended {match_id} to nodes and edges")    
    
### CLEAN NODES.CSV FILE ###

print(f"{nodes.duplicated().sum()} duplicates before drop")
nodes = nodes.drop_duplicates()
print(f"{nodes.duplicated().sum()} duplicates after drop")
    
### OUTPUT TO EDGES AND NODES TO CSV FILES ###

# Write to edges.csv and nodes.csv
edges.to_csv(f"{directory}edges.csv", mode="a", header=False, index=False)
nodes.to_csv(f"{directory}nodes.csv", mode="a", header=False, index=False)
    
print("Wrote to the csv files")
df.head()

A445182 and QL6379034
Appended QL6379034 to nodes and edges
A445182 and YG9756632
Appended YG9756632 to nodes and edges
A445182 and A637470
Appended A637470 to nodes and edges
A445182 and WP4802642
Appended WP4802642 to nodes and edges
A445182 and BA8594640
Appended BA8594640 to nodes and edges
A445182 and A195755
Appended A195755 to nodes and edges
A445182 and A927216
Appended A927216 to nodes and edges
A445182 and BQ8038683
Appended BQ8038683 to nodes and edges
A445182 and A199479
Appended A199479 to nodes and edges
A445182 and FY4162947
Appended FY4162947 to nodes and edges
A445182 and A398870
Appended A398870 to nodes and edges
A445182 and YM3561700
Appended YM3561700 to nodes and edges
A445182 and RQ3314375
Appended RQ3314375 to nodes and edges
A445182 and A684443
Appended A684443 to nodes and edges
A445182 and T626480
Appended T626480 to nodes and edges
A445182 and ZE4463791
Appended ZE4463791 to nodes and edges
A445182 and YG9844517
Appended YG9844517 to nodes and edges
A445182 

,target,name,main_cm,main_long,main_gen,match_cm,match_long,match_gen,gen_diff,tree,email,main_id,match_id,type,shared_cm
0,A865747,*wafrizzell,3587.0,263.7,1.0,29.6,23.7,4.5,3.5,GED,Wafrizzell@gmail.com,A445182,A005048,Undirected,3587.0
1,A527539,*S. C.,3587.0,281.5,1.0,29.6,23.7,4.5,3.5,,Wafrizzell@gmail.com,A445182,A005048,Undirected,3587.0
2,A089728,*D. C.,2550.1,172.2,1.2,23.9,23.9,4.6,3.4,GED,Wafrizzell@gmail.com,A445182,A005048,Undirected,2550.1
3,YG9756632,catrina Luke,50.1,28.0,4.1,24.3,24.3,4.6,0.5,,catrina77@msn.com,A445182,A005048,Undirected,50.1
4,A457247,Catrina Thomas,47.7,47.7,4.1,24.4,24.4,4.6,0.5,,catrina77@msn.com,A445182,A005048,Undirected,47.7
